In [29]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [30]:
from gensim import models
import numpy as np

In [31]:
import gensim
from gensim import corpora
from pprint import pprint

In [33]:
df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv", chunksize=10000)

In [34]:
content_en_list = []  # append the content from each chunk df here 

In [35]:
stop_words = set(stopwords.words('english'))

In [36]:
punctuation = '!"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'

df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv", chunksize=10000)

content_en_list = []  # append the content from each chunk df here 

def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    words = word_tokenize(text)
    return words

stop_words = set(stopwords.words('english'))

def remove_stopwords(word_tokens):
    words = [w for w in word_tokens if not w in stop_words]
    words = [w for w in word_tokens if not w in punctuation]
    return words


# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = chunk.drop(['Unnamed: 0'], axis=1)
    content_en = chunk.content_en
    print(type(content_en))
    content_en = content_en.apply(tokenize_column)
    content_en = content_en.apply(remove_stopwords)
    chunk_filter['tokens'] = content_en
    print(type(chunk_filter))
    print('done')
    # Once the data filtering is done, append the chunk to list
    content_en_list.append(chunk_filter)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
done
<class 'pandas.core.series.Series'>
<class 'pandas.c

In [38]:
len(content_en_list)

87

In [39]:
%%time
df_concat = pd.concat(content_en_list)

Wall time: 891 ms


In [42]:
df_concat.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length,tokens
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, i, rise, on, a, point, of, orde..."
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, i, rise, on, a, point, of, orde..."
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, mr., speaker, i, ask, that, all, rem..."
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, mr., speaker, i, ask, that, all, rem..."
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3,"[mr., speaker, i, rise, today, on, a, question..."


getting the english statements

In [45]:
politician_id = list(set(df_concat.politician_id))

In [47]:
df_concat.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length',
       'tokens'],
      dtype='object')

In [48]:
d = dict()

In [49]:
d[4] = 4

In [53]:
df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})

In [54]:
for index, row in df.iterrows():
    print(row['c1'], row['c2'])

10 100
11 110
12 120


In [55]:
df_small = df_concat[:10]

In [56]:
df_small.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length,tokens
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, i, rise, on, a, point, of, orde..."
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, i, rise, on, a, point, of, orde..."
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, mr., speaker, i, ask, that, all, rem..."
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, mr., speaker, i, ask, that, all, rem..."
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3,"[mr., speaker, i, rise, today, on, a, question..."


In [59]:
politicians = list(set(df_concat.politician_id))

In [60]:
len(politicians)

1051

In [61]:
vocab_count_dict = dict()

In [63]:
vocab_count_dict = vocab_count_dict.fromkeys(politicians, set())

In [74]:
%%time
counter = 0
for index, row in df_concat.iterrows(): 
    if counter%100000 ==0:
        print('at row', counter)
        
    pid = row['politician_id']
    vocab_count_dict[pid].update(row['tokens'])
    counter+=1

at row 0
at row 10000
at row 20000
at row 30000
at row 40000
at row 50000
at row 60000
at row 70000
at row 80000
at row 90000
at row 100000
at row 110000
at row 120000
at row 130000
at row 140000
at row 150000
at row 160000
at row 170000
at row 180000
at row 190000
at row 200000
at row 210000
at row 220000
at row 230000
at row 240000
at row 250000
at row 260000
at row 270000
at row 280000
at row 290000
at row 300000
at row 310000
at row 320000
at row 330000
at row 340000
at row 350000
at row 360000
at row 370000
at row 380000
at row 390000
at row 400000
at row 410000
at row 420000
at row 430000
at row 440000
at row 450000
at row 460000
at row 470000
at row 480000
at row 490000
at row 500000
at row 510000
at row 520000
at row 530000
at row 540000
at row 550000
at row 560000
at row 570000
at row 580000
at row 590000
at row 600000
at row 610000
at row 620000
at row 630000
at row 640000
at row 650000
at row 660000
at row 670000
at row 680000
at row 690000
at row 700000
at row 710000
at row

In [70]:
vocab_count_dict[3465].update(tokens)